In [42]:
import random
from collections import Counter

In [43]:
def roll_die() -> int:
    return random.randint(1, 6)

def is_even(n: int) -> bool:
    return n % 2 == 0

def is_divisible_by_three(n: int) -> bool:
    return n % 3 == 0

def greater_than_eight(n: int) -> bool:
    return n > 8

def less_than_eight(n: int) -> bool:
    return n < 8

def sum_values(*n: int) -> int:
    return sum(n)

In [44]:
from enum import Enum, auto

class State(Enum):
    RED = auto()
    GREEN = auto()

### Naive, Imperative Implementation

In [45]:
def game() -> State:
    value = roll_die()

    state = State.GREEN if is_even(value) else State.RED

    if state is State.GREEN:
        value = sum_values(roll_die(), roll_die())
    else:
        value = roll_die()
        value *= 2
        if greater_than_eight(value):
            state = State.GREEN
    
    if state is State.GREEN and less_than_eight(value):
        state = State.RED

    
    return state

In [46]:
results = [game() for _ in range(100_000)]
Counter(results)

Counter({<State.GREEN: 2>: 37444, <State.RED: 1>: 62556})

In [47]:
random.seed(9)
value = game()

### Naive, Functional Implementation

In [48]:
def step_1(value: int) -> State:
    return State.GREEN if is_even(value) else State.RED

def step_2(_: int) -> int:
    return sum_values(roll_die(), roll_die())

def step_3(_: int) -> int:
    return roll_die() * 2

def step_4(value: int) -> State:
    return State.GREEN if greater_than_eight(value) else State.RED

def step_5(value: int) -> State:
    return State.RED if less_than_eight(value) else State.GREEN

def game_() -> State:
    value = roll_die()

    state = step_1(value)

    if state is State.GREEN:
        value = step_2(value)
    else:
        value = step_3(value)
        state = step_4(value)
    
    if state is State.GREEN:
        state = step_5(value)
    
    return state

In [49]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_()
    assert value is value_, i

### Pure Python Implementation

In [50]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Callable, ParamSpec, Union, TypeVar, Generic

T = TypeVar("T", covariant=True)

@dataclass
class Success(Generic[T]):
    value: T

@dataclass
class Failure(Generic[T]):
    value: T


T1 = TypeVar("T1")
T2 = TypeVar("T2")
T3 = TypeVar("T3")


Result = Union[Success[T1], Failure[T2]]


U = TypeVar("U")
W = TypeVar("W")
P = ParamSpec("P")

def compose(f: Callable[P, U], g: Callable[[U], W]) -> Callable[P, W]:
    def inner(*args: P.args, **kwargs: P.kwargs) -> W:
        return g(f(*args, **kwargs))
    return inner

def invert(value: Result[T1, T2]) -> Result[T2, T1]:
    return Failure(value.value) if isinstance(value, Success) else Success(value.value)

def identity(x: T) -> T:
    return x


def bind(f: Callable[[T1], Result[T3, T2]]) -> Callable[[Result[T1, T2]], Result[Union[T1, T3], T2]]:
    def adapt(double_track: Result[T1, T2]) -> Result[Union[T1, T3], T2]:
        return f(double_track.value) if isinstance(double_track, Success) else  double_track
    return adapt

def map_(f: Callable[[T1], Success[T3]]) -> Callable[[Result[T1, T2]], Result[T3, T2]]:
    def adapt(double_track: Result[T1, T2]) -> Result[T3, T2]:
        return f(double_track.value) if isinstance(double_track, Success) else  double_track
    return adapt


# def lash(f: Callable[..., Result[T1, T3]]) -> Callable[[Result[T1, T2]], Result[T1, Union[T2, T3]]]:
#     def adapt(double_track: Result[T1, T2]) -> Result[T1, Union[T2, T3]]:
#         return f(double_track.value) if isinstance(double_track, Failure) else  double_track
#     return adapt

def lash(f: Callable[[T1], Result[T1, T3]]) -> Callable[[Result[T1, T2]], Result[T1, Union[T2, T3]]]:
    return compose(compose(invert, bind(compose(f, invert))), invert)

# def alt(f: Callable[..., Failure[T3]]) -> Callable[[Result[T1, T2]], Result[T1, T3]]:
#     def adapt(double_track: Result[T1, T2]) -> Result[T1, T3]:
#         return f(double_track.value) if isinstance(double_track, Failure) else  double_track
#     return adapt

def alt(f: Callable[[T1], Failure[T3]]) -> Callable[[Result[T1, T2]], Result[T1, T3]]:
    return compose(compose(invert, map_(compose(f, invert))), invert)


@dataclass
class PipeLazyWithStart(Generic[P, W]):
    f: Callable[P, W]

    def then(self, g: Callable[[W], U]) -> PipeLazyWithStart[P, U]:
        return PipeLazyWithStart(compose(self.f, g))

    def __call__(self, *args: P.args, **kwargs: P.kwargs) -> W:
        return self.f(*args, **kwargs)


In [51]:
def step_1(value: int) -> Result[int, int]:
    return Success(value) if is_even(value) else Failure(value)

def step_2(_: int) -> Success[int]:
    value = sum_values(roll_die(), roll_die())
    return Success(value)

def step_3(_: int) -> Failure[int]:
    value = roll_die()
    return Failure(value * 2)

def step_4(value: int) -> Result[int, int]:
    return Success(value) if greater_than_eight(value) else Failure(value)

def step_5(value: int) -> Result[int, int]:
    return Failure(value) if less_than_eight(value) else Success(value)

def map_result_to_state(value: Result[int, int]) -> State:
    return State.GREEN if isinstance(value, Success) else State.RED

random.seed(9)
game_2 = (
    PipeLazyWithStart(step_1)
    .then(map_(step_2))
    .then(alt(step_3))
    .then(lash(step_4))
    .then(bind(step_5))
    .then(map_result_to_state)
)
game_2(roll_die())

<State.GREEN: 2>

In [52]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_2(roll_die())
    # print(value, value_)
    assert value is value_, (value, value_, i)

### Implementing using Returns

In [53]:
%pip install returns mypy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\elcg\.pyenv\pyenv-win\versions\3.11.0a7\python.exe -m pip install --upgrade pip' command.


In [54]:
from returns.result import Result, Success, Failure

In [55]:
def step_1(value: int) -> Result[int, int]:
    return Success(value) if is_even(value) else Failure(value)

def step_2(_: int) -> int:
    value = sum_values(roll_die(), roll_die())
    return value

def step_3(_: int) -> int:
    value = roll_die()
    return value * 2

def step_4(value: int) -> Result[int, int]:
    return Success(value) if greater_than_eight(value) else Failure(value)

def step_5(value: int) -> Result[int, int]:
    return Failure(value) if less_than_eight(value) else Success(value)

def map_result_to_state(value: Result[int, int]) -> State:
    return State.GREEN if isinstance(value, Success) else State.RED

In [56]:
from returns.pipeline import pipe
from returns.pointfree import bind, lash, map_, alt

game_3 = pipe(
    step_1,
    map_(step_2),
    alt(step_3),
    lash(step_4),
    bind(step_5),
    map_result_to_state
)
game_3(roll_die())

<State.GREEN: 2>

In [57]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_3(roll_die())
    # print(value, value_)
    assert value is value_, i

### Implementing using Result

In [58]:
%pip install result

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\elcg\.pyenv\pyenv-win\versions\3.11.0a7\python.exe -m pip install --upgrade pip' command.


In [59]:
from result import Ok, Err, Result

In [60]:
T1 = TypeVar("T1")
T2 = TypeVar("T2")
T3 = TypeVar("T3")

def invert(value: Result[T1, T2]) -> Result[T2, T1]:
    return Err(value.unwrap()) if isinstance(value, Ok) else Ok(value.unwrap_err())

def bind(f: Callable[..., Result[T3, T2]]) -> Callable[[Result[T1, T2]], Result[Union[T1, T3], T2]]:
    def adapt(double_track: Result[T1, T2]) -> Result[Union[T1, T3], T2]:
        return f(double_track.unwrap()) if double_track.is_ok() else  double_track
    return adapt

def map_(f: Callable[..., Ok[T3]]) -> Callable[[Result[T1, T2]], Result[T3, T2]]:
    def adapt(double_track: Result[T1, T2]) -> Result[T3, T2]:
        return f(double_track.unwrap()) if isinstance(double_track, Ok) else  double_track
    return adapt

def lash(f: Callable[..., Result[T1, T3]]) -> Callable[[Result[T1, T2]], Result[T1, Union[T2, T3]]]:
    return compose(compose(invert, bind(compose(f, invert))), invert)


def alt(f: Callable[..., Err[T3]]) -> Callable[[Result[T1, T2]], Result[T1, T3]]:
    return compose(compose(invert, map_(compose(f, invert))), invert)

In [61]:
def step_1(value: int) -> Result[int, int]:
    return Ok(value) if is_even(value) else Err(value)

def step_2(_: int) -> Ok[int]:
    value = sum_values(roll_die(), roll_die())
    return Ok(value)

def step_3(_: int) -> Err[int]:
    value = roll_die()
    return Err(value * 2)

def step_4(value: int) -> Result[int, int]:
    return Ok(value) if greater_than_eight(value) else Err(value)

def step_5(value: int) -> Result[int, int]:
    return Err(value) if less_than_eight(value) else Ok(value)

def map_result_to_state(value: Result[int, int]) -> State:
    return State.GREEN if value.is_ok() else State.RED

In [62]:
game_4 = (
    PipeLazyWithStart(step_1)
    .then(map_(step_2))
    .then(alt(step_3))
    .then(lash(step_4))
    .then(bind(step_5))
    .then(map_result_to_state)
)

game_4(roll_die())

<State.GREEN: 2>

In [63]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_4(roll_die())
    # print(value, value_)
    assert value is value_, i

### Probability of Winning

In [64]:
%pip install sympy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\elcg\.pyenv\pyenv-win\versions\3.11.0a7\python.exe -m pip install --upgrade pip' command.


In [65]:
from sympy.stats import P, E, variance, Die, Normal

In [66]:
from sympy import simplify

In [67]:
X = Die("X", 6)
even = P(X % 2 <= 0)

D1, D2 = Die("D_1", 6), Die("D_2", 6)
sum_greater_than_eight = P(D1 + D2 >= 8)

D3 = Die("D_1", 6)
value_greater_than_eight = P(D3 * 2 > 8)
value_greater_than_ten = P(D3 * 2 > 10)


In [68]:
green_probability = (
           even  * sum_greater_than_eight
    + (1 - even) * value_greater_than_eight
)
green_probability

3/8

In [69]:
3 / 8

0.375